In [1]:
%cd semantic_code_search/code_search/modeling

/teamspace/studios/this_studio/semantic_code_search/code_search/modeling


In [2]:
from omegaconf import OmegaConf
from enum import Enum, auto
from abc import ABC, abstractmethod
from typing import Type
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, PreTrainedTokenizerBase,PreTrainedTokenizer
from transformers.data import DataCollatorForSeq2Seq
from peft import (PeftModel, PeftConfig, PrefixTuningConfig, 
                 LoraConfig, get_peft_model, TaskType)
from tokenizers import Tokenizer
from omegaconf import DictConfig
from omegaconf import OmegaConf
from typing import Optional, Dict, Type
from optimizers import *
from model_manager import *
from losses import *
import pandas as pd

In [3]:
model = Model("jinaai/jina-embeddings-v2-base-code")

/home/zeus/.cache/huggingface/modules/transformers_modules/jinaai/jina-bert-v2-qk-post-norm/3baf9e3ac750e76e8edd3019170176884695fb94/configuration_bert.py:29: UserWarning: optimum is not installed. To use OnnxConfig and BertOnnxConfig, make sure that `optimum` package is installed
  warnings.warn("optimum is not installed. To use OnnxConfig and BertOnnxConfig, make sure that `optimum` package is installed")


In [6]:
model.config.hidden_size

128

In [4]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

In [29]:
tokenizer(["hello world","hello"], return_tensors="pt",padding="longest")

{'input_ids': tensor([[ 101, 7592, 2088,  102],
        [ 101, 7592,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0],
        [0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])}

In [43]:
loss = InfoNCELoss(bank_size=1024,embeddings_dim=128,temperature=0.07)

In [80]:
d1 = torch.randn(128,128)
d2 = torch.randn(150,128)

In [82]:
loss.bank.ptr

150

In [81]:
loss(d1,d2)

tensor(7.8471)